In [ ]:
import numpy as np
import cv2
from PIL import Image, ImageDraw
import keras
from keras import backend as K
from keras.models import load_model
import tkinter as tk
from PIL import Image, ImageTk, ImageDraw
import threading
import time
import queue
import re

print("Loading...")

K.image_data_format()

print("Done")

# Carica il modello
model = load_model('pre-trained-models/eq_solver4.keras')

class DrawingApp:
    def __init__(self, master):
        self.master = master
        self.drawing_color = "black"
        self.eraser_color = "white"
        self.current_color = self.drawing_color
        self.old_x = None
        self.old_y = None
        self.penwidth = 5
        self.eraserwidth = 20
        self.current_width = self.penwidth
        self.drawable_area = tk.Canvas(self.master, width=1400, height=700, bg='white')
        self.drawable_area.pack()
        self.drawable_area.bind('<B1-Motion>', self.paint)
        self.drawable_area.bind('<ButtonRelease-1>', self.reset)
        
        self.image = Image.new("RGB", (800, 800), color="white")
        self.draw = ImageDraw.Draw(self.image)
        
        self.equation_label = tk.Label(self.master, text="Equation: ")
        self.equation_label.pack()
        
        self.result_label = tk.Label(self.master, text="Result: ")
        self.result_label.pack()
        
        self.clear_button = tk.Button(self.master, text="Clear All", command=self.clear)
        self.clear_button.pack(side=tk.LEFT)
        
        self.eraser_button = tk.Button(self.master, text="Eraser", command=self.use_eraser)
        self.eraser_button.pack(side=tk.LEFT)
        
        self.pen_button = tk.Button(self.master, text="Pen", command=self.use_pen)
        self.pen_button.pack(side=tk.LEFT)
        
        self.equation_queue = queue.Queue()
        self.update_thread = threading.Thread(target=self.update_equation, daemon=True)
        self.update_thread.start()
        
        self.master.after(100, self.check_queue)

    def paint(self, event):
        if self.old_x and self.old_y:
            self.drawable_area.create_line(self.old_x, self.old_y, event.x, event.y, 
                                           width=self.current_width, fill=self.current_color,
                                           capstyle=tk.ROUND, smooth=tk.TRUE, splinesteps=36)
            self.draw.line([self.old_x, self.old_y, event.x, event.y], 
                           fill=self.current_color, width=self.current_width)
        self.old_x = event.x
        self.old_y = event.y

    def reset(self, event):
        self.old_x, self.old_y = None, None

    def clear(self):
        self.drawable_area.delete("all")
        self.image = Image.new("RGB", (1400, 700), color="white")
        self.draw = ImageDraw.Draw(self.image)
        self.equation_label.config(text="Equation: ")
        self.result_label.config(text="Result: ")

    def use_eraser(self):
        self.current_color = self.eraser_color
        self.current_width = self.eraserwidth

    def use_pen(self):
        self.current_color = self.drawing_color
        self.current_width = self.penwidth

    def update_equation(self):
        while True:
            try:
                img = np.array(self.image.convert('L'))
                equation = self.process_image(img)
                self.equation_queue.put(equation)
            except Exception as e:
                print(f"Error in update_equation: {e}")
            time.sleep(0.5)

    def check_queue(self):
        try:
            equation = self.equation_queue.get_nowait()
            if equation:  # Verifica se l'equazione non è vuota
                self.equation_label.config(text=f"Equation: {equation}")
                result = self.solve_equation(equation)
                self.result_label.config(text=f"Result: {result}")
            else:
                self.equation_label.config(text="Equation: None detected")
                self.result_label.config(text="Result: N/A")
        except queue.Empty:
            pass
        except Exception as e:
            print(f"Error in check_queue: {e}")
        finally:
            self.master.after(100, self.check_queue)

    def process_image(self, img):
        img = ~img
        _, thresh = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY)
        ctrs, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
        cnt = sorted(ctrs, key=lambda ctr: cv2.boundingRect(ctr)[0])
        w, h = 28, 28
        train_data = []
        rects = []
        for c in cnt:
            x, y, w, h = cv2.boundingRect(c)
            rect = [x, y, w, h]
            rects.append(rect)
        
        bool_rect = []
        for r in rects:
            l = []
            for rec in rects:
                flag = 0
                if rec != r:
                    if (r[0] < (rec[0] + rec[2] + 10) and rec[0] < (r[0] + r[2] + 10) and
                        r[1] < (rec[1] + rec[3] + 10) and rec[1] < (r[1] + r[3] + 10)):
                        flag = 1
                l.append(flag)
            if rec == r:
                l.append(0)
            bool_rect.append(l)
        dump_rect = []
        for i in range(len(cnt)):
            for j in range(len(cnt)):
                if bool_rect[i][j] == 1:
                    area1 = rects[i][2] * rects[i][3]
                    area2 = rects[j][2] * rects[j][3]
                    if area1 == min(area1, area2):
                        dump_rect.append(rects[i])
        
        final_rect = [i for i in rects if i not in dump_rect]
        for r in final_rect:
            x, y, w, h = r
            im_crop = thresh[y:y+h+10, x:x+w+10]
            im_resize = cv2.resize(im_crop, (28, 28))
            im_resize = np.reshape(im_resize, (28, 28, 1))
            train_data.append(im_resize)

        equation = ''
        for i in range(len(train_data)):
            train_data[i] = np.array(train_data[i])
            train_data[i] = train_data[i].reshape(1, 28, 28, 1)
            result = np.argmax(model.predict(train_data[i]), axis=-1)

            if result[0] < 10:
                equation += str(result[0])
            elif result[0] == 10:
                equation += "+"
            elif result[0] == 11:
                equation += "-"
            elif result[0] == 12:
                equation += "*"
            elif result[0] == 13:
                equation += "/"
            elif result[0] == 14:
                equation += "="
            elif result[0] == 15:
                equation += "."
            elif result[0] == 16:
                equation += "x"
            elif result[0] == 17:
                equation += "y"
            elif result[0] == 18:
                equation += "z"

        if not equation:
            return None
        return equation

    def solve_equation(self, equation):
        if not equation:
            return "No equation detected"

        try:
            if "=" in equation:
                left_side, right_side = equation.split('=')

                # Verifica se la parte sinistra è la definizione di una variabile (ad es. "y =")
                if left_side.strip() in ['x', 'y', 'z']:  # Variabili considerate
                    variable_name = left_side.strip()

                    # Cerchiamo un'assegnazione diretta del valore alla variabile
                    direct_assignment = re.search(r'(\d+(\.\d+)?)', right_side)
                    
                    if direct_assignment:
                        # Se troviamo un'assegnazione diretta, usiamo quel valore
                        variable_value = float(direct_assignment.group(1))
                    else:
                        # Altrimenti, cerchiamo di risolvere l'equazione per la variabile
                        try:
                            # Rimuoviamo la variabile dal lato destro
                            right_side_without_var = re.sub(r'\b' + variable_name + r'\b', '', right_side)
                            # Risolviamo l'equazione
                            variable_value = eval(right_side_without_var)
                        except:
                            return f"Impossibile calcolare il valore di {variable_name}"

                    if variable_value is not None:
                        # Ora possiamo usare questo valore nell'espressione originale
                        right_side_cleaned = re.sub(r'\d*\.?\d*\*?' + variable_name, variable_name, right_side)
                        try:
                            new_expression = right_side_cleaned.replace(variable_name, str(variable_value))
                            # Sostituiamo il punto decimale con una virgola per evitare problemi con eval()
                            new_expression = new_expression.replace(',', '.')
                            final_result = eval(new_expression)
                            return f"{variable_name} = {variable_value}, Risultato finale: {final_result}"
                        except Exception as e:
                            return f"Errore durante la valutazione della nuova espressione: {e}"
                    else:
                        return "Impossibile calcolare il risultato finale."
                else:
                    try:
                        # Se non è una definizione di variabile, calcola normalmente
                        result = eval(left_side)
                        return f"Il risultato dell'equazione è: {result}"
                    except Exception as e:
                        return f"Errore durante la valutazione dell'equazione: {e}"
            else:
                # Se non c'è un segno di uguale, calcola direttamente l'espressione
                try:
                    result = eval(equation)
                    return f"Il risultato dell'espressione è: {result}"
                except Exception as e:
                    return f"Errore durante la valutazione dell'espressione: {e}"
        except Exception as e:
            return f"Errore generico: {e}"

if __name__ == "__main__":
    root = tk.Tk()
    app = DrawingApp(root)
    root.mainloop()

Loading...
Done
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/ste

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 145ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 131ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 118ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 141ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
